# **Extracting the zip file using python**

In [ ]:
import zipfile
zip_ref = zipfile.ZipFile('/content/PlantVillage.zip')
zip_ref.extractall('/content')
zip_ref.close()

# **Importing the Libraries**

In [ ]:
import tensorflow
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense,Conv2D,Flatten
import matplotlib.pyplot as plt

In [ ]:
dataset = keras.preprocessing.image_dataset_from_directory(
'/content/PlantVillage',
   image_size = (256,256),
    shuffle = True,
    batch_size = 32
)

In [ ]:
class_names = dataset.class_names

In [ ]:
len(dataset)

In [ ]:
for img_batch,label_batch in dataset.take(1):
  print(img_batch.shape)
  print(label_batch.numpy())

In [ ]:
plt.figure(figsize=(10,10))
for img_batch,label_batch in dataset.take(1):
  for i in range(12):
    ax = plt.subplot(4,3,i+1)
    plt.imshow(img_batch[i].numpy().astype('uint8'))
    plt.title(class_names[label_batch[i]])
    plt.axis("off")

In [ ]:
train_size = 0.8
len(dataset)*train_size

In [ ]:
train_ds = dataset.take(54)
len(train_ds)


In [ ]:
test_ds = dataset.skip(54)
len(test_ds)


In [ ]:
val_size=0.1
len(dataset)*val_size

In [ ]:
val_ds = test_ds.take(6)
len(val_ds)

In [ ]:
test_ds = test_ds.skip(6)
len(test_ds)

In [ ]:
def get_dataset_partitions_tf(ds, train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=10000):
    assert (train_split + test_split + val_split) == 1

    ds_size = len(ds)

    if shuffle:
        ds = ds.shuffle(shuffle_size, seed=12)

    train_size = int(train_split * ds_size)
    val_size = int(val_split * ds_size)

    train_ds = ds.take(train_size)
    val_ds = ds.skip(train_size).take(val_size)
    test_ds = ds.skip(train_size).skip(val_size)

    return train_ds, val_ds, test_ds


In [ ]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)

In [ ]:
print(len(train_ds))
print(len(test_ds))
print(len(val_ds))

In [ ]:
train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=tensorflow.data.AUTOTUNE)
val_ds = val_ds.cache().shuffle(1000).prefetch(buffer_size=tensorflow.data.AUTOTUNE)
test_ds = test_ds.cache().shuffle(1000).prefetch(buffer_size=tensorflow.data.AUTOTUNE)

# Resizing and Rescaling

In [ ]:
resize_rescale = keras.Sequential([
    keras.layers.experimental.preprocessing.Resizing(256,256),
    keras.layers.experimental.preprocessing.Rescaling(1.0/255)
])

# **Data Augmentation**

In [ ]:
data_augmentation = keras.Sequential([
    keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
    keras.layers.experimental.preprocessing.RandomRotation(0.2)
])

In [ ]:
from keras import layers
model = Sequential([
    resize_rescale,
    layers.Conv2D(32, kernel_size = (3,3), activation='relu', input_shape=(32,256,256,3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64,  kernel_size = (3,3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(3, activation='softmax'),
])

model.build(input_shape=(32,256,25,3))


In [ ]:
model.summary()

In [ ]:
model.compile(
    optimizer='adam',
    loss=tensorflow.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [ ]:
history = model.fit(
    train_ds,
    batch_size=32,
    validation_data=val_ds,
    verbose=1,
    epochs=20,
)

In [ ]:
scores = model.evaluate(test_ds)

In [ ]:
plt.plot(history.history['accuracy'],label='train')
plt.plot(history.history["val_accuracy"],label='validation')
plt.legend()
plt.show()

In [ ]:
plt.plot(history.history['loss'],color='red',label='train')
plt.plot(history.history['val_loss'],color='blue',label='validation')
plt.legend()
plt.show()

In [ ]:
import numpy as np
for image_batch, label_batch in test_ds.take(1):
  first_img = image_batch[0].numpy().astype('uint8')
  first_label  = label_batch[0].numpy()
  print("First image to predict")
  plt.imshow(first_img)
  print("Actual Image:",class_names[first_label])

  batch_prediction = model.predict(image_batch)
  print("Predicted_label",np.argmax(batch_prediction[0]))

In [ ]:
def predict(model, img):
    img_array = tensorflow.keras.preprocessing.image.img_to_array(images[i].numpy())
    img_array = tensorflow.expand_dims(img_array, 0)

    predictions = model.predict(img_array)

    predicted_class = class_names[np.argmax(predictions[0])]
    confidence = round(100 * (np.max(predictions[0])), 2)
    return predicted_class, confidence

In [ ]:
plt.figure(figsize=(15, 15))
for images, labels in test_ds.take(1):
    for i in range(9):
        ax = plt.subplot(3, 3, i + 1)
        plt.imshow(images[i].numpy().astype("uint8"))

        predict_class, confidence = predict(model, images[i].numpy())
        actual_class = class_names[labels[i]]

        plt.title(f"Actual: {actual_class},\n Predicted: {predict_class}.\n Confidence: {confidence}%")

        plt.axis("off")

In [ ]:
model.save("potatoes")